# Funciones desarrolladas

1.2. 🛠️ Instalación

### 🔧Carga de librerias en el entorno

In [1]:
# Carga de CSV
import pandas as pd

# Funciones de carga de ficheros
import os
# Para generar el gráfico de dependecias
import networkx as nx
import matplotlib.pyplot as plt
# Calculo de localizaciones
import math
import geocoder

import inspect


import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt

import re

## 💻 Funciones sobre el sistema

### get_environment_info 

Esta función detecta si el entorno donde se ejecuta el código es:

🐳 Un contenedor Docker

💻 Una máquina virtual (VM)

🖥️ Un sistema físico (bare-metal)



In [12]:
def get_environment_info():
    import os
    import platform
    import subprocess

    def is_docker():
        try:
            if os.path.exists("/.dockerenv"):
                return True
            with open("/proc/1/cgroup", "r") as f:
                if "docker" in f.read() or "containerd" in f.read():
                    return True
        except Exception:
            pass
        return False

    def is_virtual_machine():
        system = platform.system()

        try:
            if system == "Linux":
                try:
                    output = subprocess.check_output(["systemd-detect-virt"], stderr=subprocess.DEVNULL).decode().strip()
                    if output and output != "none":
                        return True, output
                except Exception:
                    pass

                try:
                    with open("/proc/cpuinfo", "r") as f:
                        if "hypervisor" in f.read().lower():
                            return True, "Desconocido (hypervisor detectado)"
                except Exception:
                    pass

                try:
                    output = subprocess.check_output(["dmidecode", "-s", "system-product-name"], stderr=subprocess.DEVNULL).decode().strip().lower()
                    for vm in ["virtualbox", "kvm", "vmware", "microsoft"]:
                        if vm in output:
                            return True, vm.capitalize()
                except Exception:
                    pass

            elif system == "Windows":
                try:
                    ps_script = "Get-CimInstance -ClassName Win32_ComputerSystem | Select-Object -ExpandProperty Model"
                    output = subprocess.check_output(["powershell", "-Command", ps_script], stderr=subprocess.DEVNULL).decode().strip().lower()
                    for vm in ["virtualbox", "kvm", "vmware", "hyper-v", "virtual"]:
                        if vm in output:
                            return True, vm.capitalize()
                except Exception:
                    pass

        except Exception:
            pass

        return False, None

    docker = is_docker()
    vm, vm_type = is_virtual_machine()

    if docker:
        entorno = "docker"
    elif vm:
        entorno = "virtual"
    else:
        entorno = "real"

    return {
        "docker": docker,
        "virtual_machine": vm,
        "hypervisor": vm_type if vm else False,
        "entorno": entorno
    }

# Ejemplo de uso
env_info = get_environment_info()
print(env_info)

if env_info["entorno"] == "Docker":
    print("🐳 Ejecutándose dentro de un contenedor Docker")
elif env_info["entorno"] == "virtual":
    print(f"💻 Ejecutándose en máquina virtual ({env_info['hypervisor']})")
else:
    print("🖥️ Ejecutándose en máquina física")



{'docker': False, 'virtual_machine': False, 'hypervisor': False, 'entorno': 'real'}
🖥️ Ejecutándose en máquina física


### clean_file

🔍 ¿Qué hace?

Esta función recorre todos los archivos dentro de un **directorio (y sus subdirectorios)** y limpia los saltos de línea Windows (**\r\n**), reemplazándolos por saltos de línea estándar de Unix (\n), en archivos que tengan extensiones específicas (*por defecto, .txt*).

🧱 ¿Qué hace paso a paso?

Recorrer el directorio y subdirectorios. 

La función filtra los archivos para solo procesar aquellos que tienen extensiones que coinciden con las proporcionadas en el parámetro extensions.

Abrir y leer el contenido del archivo:

Limpiar el contenido del archivo reemplaza los saltos de línea \r\n (utilizados en sistemas Windows) por \n (salto de línea estándar en sistemas Unix/Linux).


Después de limpiar el contenido y se sobrescribe con el contenido limpio.

Manejo de errores:

Si ocurre un error durante la lectura o escritura del archivo, se captura con un bloque try-except y se imprime un mensaje de error, pero el proceso continúa con el siguiente archivo.

Si ocurre un error al recorrer los directorios o abrir el archivo principal, se devuelve un mensaje de error indicando la excepción.


In [7]:
def clean_file(directory, extensions=['.txt']):
    try : 
      for root, dirs, files in os.walk(directory):  # Recorrer directorios y subdirectorios
        for file_name in files:
            # print('extension:',extensions)
            if any(file_name.endswith(extension) for extension in extensions):  # Filtra por extensiones
                file_path = os.path.join(root, file_name)  # Obtiene la ruta completa
                # print(f'Procesando archivo: {file_path}')
                
                try:
                    with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                        content = f.read()

                    content_cleaned = content.replace('\r\n', '\n')

                    with open(file_path, 'w', encoding='utf-8') as f:
                        f.write(content_cleaned)

                    # print(f'✔ Procesado: {file_name}')

                except Exception as e:
                    print(f'❌ Error al procesar {file_name}: {e}. Se omitirá.')
    except Exception:
        return f'error {Exception}'
        
        

In [8]:
%matplotlib inline

## Importación de Librerias

## 📊 De tratamiento de las entidades del dataframe

### lowercase_if_letters_only

🔍 ¿Qué hace?

Función para convertir a minúsculas solo las columnas con letras


🧱 ¿Qué hace paso a paso?

    1. Recorre todas las columnas del DataFrame.
    2. Para cada columna:
       - Convierte cada valor a cadena.
       - Verifica si contiene solo letras. (para evitar tocar las columnas que tienen numeros
    3. Si todos los valores de la columna son letras:
       - Convierte la columna completa a minúsculas usando.
    4. Devuelve el DataFrame modificado.

In [9]:
def lowercase_if_letters_only(df_temp):
    try: 	
        for col in df_temp.columns:
            if df_temp[col].apply(lambda x: str(x).isalpha()).all():
                df_temp[col] = df_temp[col].str.lower()
        return df_temp
    except Exception:
        return f'error {Exception}'


# Ejemplo
data = {
    'Nombre': ['ALICe', 'BOB', 'CAROL'],
    'Edad': [25, 30, 22],
    'Ciudad': ['MADRID', 'SEVILLA', 'VALENCIA']
}
'''
df = pd.DataFrame(data)

# Aplicamos la función
df_modificado = lowercase_if_letters_only(df)

print(df_modificado)
'''




'\ndf = pd.DataFrame(data)\n\n# Aplicamos la función\ndf_modificado = lowercase_if_letters_only(df)\n\nprint(df_modificado)\n'

### is_words(column)

 🔍 ¿Qué hace?

 Verifica si todos los valores de una columna son solo letras (sin números ni caracteres especiales)

In [6]:
def is_words(column):
   return column.apply(lambda x: bool(re.match(r'^[A-Za-z]+$', str(x)))).all()

### lower_column

🔍 ¿Qué hace?

Verifica si todos los valores en una serie contienen solo letras (ignorando strings vacíos).


In [11]:
def lower_column(df_temp, columns_str):
    for column in columns_str:
        if column in df_temp.columns:
            # Convertir la columna a tipo string, manejando nulos
            df_temp[column] = df_temp[column].fillna('').astype(str)
            
            # Verificar si todos los valores son letras
            if is_words(df_temp[column]):
                # Si todos los valores son letras, convertir a minúsculas
                df_temp[column] = df_temp[column].str.lower()

    return df_temp
'''
### Ejemplo de uso:

data = {
    'Nombre': ['ALICE', 'BOB', 'CAROL'],
    'Ciudad': ['MADRID', 'SEVILLA', 'VALENCIA'],
    'Edad': [25, 30, 22]  # Esta columna no debe cambiar
}

df = pd.DataFrame(data)

# Aplicamos la función
df_mod = lower_column(df, ['Nombre', 'Ciudad', 'Edad'])

print(df_mod)
'''

"\n### Ejemplo de uso:\n\ndata = {\n    'Nombre': ['ALICE', 'BOB', 'CAROL'],\n    'Ciudad': ['MADRID', 'SEVILLA', 'VALENCIA'],\n    'Edad': [25, 30, 22]  # Esta columna no debe cambiar\n}\n\ndf = pd.DataFrame(data)\n\n# Aplicamos la función\ndf_mod = lower_column(df, ['Nombre', 'Ciudad', 'Edad'])\n\nprint(df_mod)\n"

### load_files_in_dataframes

🔍 ¿Qué hace?

La función load_files_in_dataframes carga todos los archivos .csv o .txt de un directorio (y sus subdirectorios) en DataFrames de pandas, realizando varias transformaciones específicas.

🧱 Paso a paso
* Inicializa listas y diccionarios:

lista_id: almacena nombres de columnas que terminan en _id o empiezan por id.

dataframes: contiene los DataFrames con claves basadas en el nombre del archivo.

🔹  Recorre los archivos del directorio:

* Ignora subdirectorios (aunque los recorre recursivamente).

🔹 Filtra archivos por extensión (.csv o .txt).

* Procesa cada archivo válido:

* Lee el archivo como DataFrame con pandas.read_csv().

* Convierte los nombres de columnas a minúsculas.

* Renombra columnas usando convert_list.

* Realiza slicing o manipulación de columnas definidas en convert_keys.

* Si existe la columna objectid, la usa como índice.

🔹 Limpieza de datos:

* Detecta columnas que contengan solo letras (excepto las columnas con id) y las convierte a minúsculas con lower_column.

🔹 Guarda el resultado:

* Crea un DataFrame con nombre df_nombre_del_archivo.

* También lo guarda en el diccionario dataframes.

🔹 Manejo de errores:

* Ignora archivos vacíos o con errores de análisis (pandas.errors.EmptyDataError o ParserError).

* Captura otros errores inesperados sin interrumpir el procesamiento.




In [13]:
def load_files_in_dataframes(file_path, sep=',', extensions=['.txt', '.csv']):
    list_id = []
    dataframes = {}

    try:
        for full_file_name in os.listdir(file_path):
            filepath = os.path.join(file_path, full_file_name)
            
            # Evitar procesar directorios
            if not os.path.isfile(filepath):
                load_files_in_dataframes(filepath, sep=sep, extensions=extensions)
                continue

            if any(full_file_name.endswith(extension) for extension in extensions):
                file_name = full_file_name[:-4]  # nombre sin extensión
                print(f'Procesando archivo: {full_file_name}')
                try:
                    df = pd.read_csv(filepath, sep=sep, header=0)
                    # normaliza los nombres de las columnas a minusulas
                    df.columns = df.columns.str.lower()
                    
                    # Detectar columnas ID
                    list_id_ = [col for col in df.columns if col.endswith('_id')]
                    list_id += [col for col in df.columns if col.startswith('id')] 
                    list_id = list(set(lista_id + lista_id_))
                   
                    if list_id:
                        print(f'df_{file_name} lista: {list_id}')

                    df_name = f"df_{file_name}"
                    globals()[df_name] = df
                    dataframes[file_name] = df

                except pd.errors.EmptyDataError:
                    print(f"Advertencia: El archivo {full_file_name} está vacío. Se omitirá.")
                except pd.errors.ParserError as e:
                    print(f"Error al analizar el archivo {full_file_name}: {e}. Se omitirá.")
                except Exception as e:
                    print(f"Error inesperado al procesar {full_file_name}: {e}. Se omitirá.")

        return dataframes, lista_id

    except FileNotFoundError:
        print(f"Error: El directorio '{file_path}' no existe.")
        return None, None
    except Exception as e:
        print(f"Error inesperado: {e}")
        return None, None



### get_unique_columns

🔍 La función identifica qué columnas de un DataFrame tienen valores únicos (es decir, sin repeticiones) y devuelve un diccionario con:

🔹 El nombre de la variable del DataFrame (detectado automáticamente).

🔹 Una lista de las columnas que tienen solo valores únicos.

🧱  Paso a paso


🔹 Usa inspect para obtener el nombre de la variable del DataFrame pasado como argumento.

🔹 Identificación de columnas únicas:

    Recorre todas las columnas y usa .is_unique para saber si todos los valores son distintos en esa columna.

🔹 Devuelve un diccionario con:

    Clave: nombre del DataFrame.

    Valor: lista de columnas con valores únicos.

In [16]:
def get_unique_columns(df):
    """
    Retorna un diccionario con el nombre de la variable del DataFrame y sus columnas con valores únicos.
    """
    # Inspeccionar el stack para encontrar el nombre de la variable pasada como argumento
    callers_local_vars = inspect.currentframe().f_back.f_locals.items()
    df_name = next((name for name, val in callers_local_vars if val is df), "dataframe")

    unique_cols = [col for col in df.columns if df[col].is_unique]
    return {df_name: unique_cols}


    ## Ejemplo



data = {
    'id': [1, 2, 3, 4],
    'nombre': ['Ana', 'Luis', 'Ana', 'Pedro'],
    'email': ['a@example.com', 'b@example.com', 'c@example.com', 'd@example.com'],
    'edad': [25, 30, 25, 40]
}
'''
df = pd.DataFrame(data)

# Obtener columnas únicas
columnas_unicas = get_unique_columns(df)

print("Columnas con valores únicos:", columnas_unicas)
'''


'\ndf = pd.DataFrame(data)\n\n# Obtener columnas únicas\ncolumnas_unicas = get_unique_columns(df)\n\nprint("Columnas con valores únicos:", columnas_unicas)\n'

## Dibujar graficos

### draw_graf

🔍 ¿Qué hace?

Es una función para visualizar un grafo utilizando NetworkX para la estructura del grafo y Matplotlib para la visualización. 

🧱 Explicación paso a paso

Los nodos están posicionados de manera automática utilizando el algoritmo de disposición de primavera.

Los nodos tienen un tamaño y color definidos.

Las etiquetas de los nodos muestran un atributo personalizado ('columnas_comunes').

Las aristas son flechas (porque el grafo es dirigido).

El grafo tiene un título que es pasado como parámetro.

Es ideal para representar estructuras de datos, redes de relaciones o dependencias entre elementos (como en análisis de grafos de dependencias o redes de información).

In [17]:
def draw_graf(grafo, titulo, color):
    plt.figure(figsize=(12, 10))
    pos = nx.spring_layout(grafo, k=2)
    nx.draw(grafo, pos, with_labels=True, node_size=5000, node_color=color, font_size=10, font_weight='bold', arrows=True)
    labels = nx.get_node_attributes(grafo, 'columnas_comunes')
    nx.draw_networkx_labels(grafo, pos, labels=labels, font_size=8, font_color='black')
    plt.title(titulo)
    plt.show()

'''
    # Ejemplo
# Crear un grafo dirigido
grafo = nx.DiGraph()

# Agregar nodos con un atributo 'columnas_comunes'
grafo.add_node("df1", columnas_comunes=["col1", "col2", "col3"])
grafo.add_node("df2", columnas_comunes=["col2", "col4"])
grafo.add_node("df3", columnas_comunes=["col1", "col5"])
grafo.add_node("df4", columnas_comunes=["col6", "col2"])

# Agregar aristas entre los nodos para representar dependencias
grafo.add_edges_from([("df1", "df2"), ("df1", "df3"), ("df2", "df4"), ("df3", "df4")])

# Llamar a la función para dibujar el grafo
draw_graf(grafo, "Ejemplo de Grafo de Dependencias", color='lightblue')
'''

'\n    # Ejemplo\n# Crear un grafo dirigido\ngrafo = nx.DiGraph()\n\n# Agregar nodos con un atributo \'columnas_comunes\'\ngrafo.add_node("df1", columnas_comunes=["col1", "col2", "col3"])\ngrafo.add_node("df2", columnas_comunes=["col2", "col4"])\ngrafo.add_node("df3", columnas_comunes=["col1", "col5"])\ngrafo.add_node("df4", columnas_comunes=["col6", "col2"])\n\n# Agregar aristas entre los nodos para representar dependencias\ngrafo.add_edges_from([("df1", "df2"), ("df1", "df3"), ("df2", "df4"), ("df3", "df4")])\n\n# Llamar a la función para dibujar el grafo\ndraw_graf(grafo, "Ejemplo de Grafo de Dependencias", color=\'lightblue\')\n'

### dependency_dataframes

🔍 ¿Qué hace?

Analiza los DataFrames, crea grafos de dependencias y los visualiza.

    Args:
    
 🔹 globals_dict: Diccionario con variables globales (como globals()).
 
 🔹 split_key: Subcadena para separar DataFrames en dos grupos.

🧱 Explicación paso a paso

🔹 Clasifica las relaciones según si uno de los DataFrames contiene un prefijo como "df_M4".

🔹 Dibuja un grafo visual para mostrar cómo están conectados.

🔹 Devuelve toda esa información para poder usarla luego.


In [19]:
def dependency_dataframes(globals_dict, split_key="df_M4"):
    

    relaciones = {}
    relaciones_M4 = {}
    relaciones_NO_M4 = {}

    G = nx.DiGraph()
    G_NO_M4 = nx.DiGraph()
    G_M4 = nx.DiGraph()

    df_names = [name for name in globals_dict if name.startswith('df_') and isinstance(globals_dict.get(name), pd.DataFrame)]

    if not df_names:
        return {}, {}, {}, G, G_M4, G_NO_M4

    for dfx_name in df_names:
        try:
            df1 = globals_dict[dfx_name]
            columnas_df1 = list(df1.columns)

            for df2_name in df_names:
                if df2_name != dfx_name:
                    df2 = globals_dict[df2_name]
                    columnas_df2 = list(df2.columns)
                    columnas_comunes = list(set(columnas_df1) & set(columnas_df2))

                    if columnas_comunes:
                        if df2_name.startswith(split_key) or dfx_name.startswith(split_key):
                            relaciones_M4[f"{dfx_name} - {df2_name}"] = columnas_comunes
                            G_M4.add_edge(dfx_name, df2_name)
                        else:
                            relaciones_NO_M4[f"{dfx_name} - {df2_name}"] = columnas_comunes
                            G_NO_M4.add_edge(dfx_name, df2_name)

                        G.add_edge(dfx_name, df2_name)
                        relaciones[f"{dfx_name} - {df2_name}"] = columnas_comunes
        except KeyError as e:
            print(f"Error al acceder al DataFrame '{e}': Verifica que el DataFrame esté definido.")
            continue

    # Dibujar grafos
    plt.figure(figsize=(15, 10))
    pos = nx.spring_layout(G, k=0.8)

    nx.draw(G_M4, pos, with_labels=True, node_size=4000, node_color='lightblue',
            font_size=10, font_weight='bold', edge_color='blue', arrows=True)
    nx.draw(G_NO_M4, pos, with_labels=True, node_size=4000, node_color='green',
            font_size=10, font_weight='bold', edge_color='lightgreen', arrows=True)

    plt.title("Grafo de Dependencias entre DataFrames")
    plt.show()

    # Devolver resultados
    return relaciones, relaciones_M4, relaciones_NO_M4, G, G_M4, G_NO_M4

# Ejemplo 
'''
rel, rel_M4, rel_no_M4, G, G_M4, G_no_M4 = dependency_dataframes(globals())

print("Todas las relaciones:", rel)
# print("Relaciones con df_M4:", rel_M4)
# print("Relaciones sin df_M4:", rel_no_M4)
'''

'\nrel, rel_M4, rel_no_M4, G, G_M4, G_no_M4 = dependency_dataframes(globals())\n\nprint("Todas las relaciones:", rel)\n# print("Relaciones con df_M4:", rel_M4)\n# print("Relaciones sin df_M4:", rel_no_M4)\n'

### tree_dataframes

🔍 ¿Qué hace?

Analiza los DataFrames, crea un gráfico de arborescencia de dependencias y lo visualiza.

Args:

🔹globals_dict: Diccionario con variables globales (como globals()).
        
🔹split_key: Subcadena para separar DataFrames en dos grupos.

🧱 ¿Qué hace paso a paso?


In [20]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt

def tree_dataframes(globals_dict, split_key="df_M4"):
    relaciones = {}
    G = nx.DiGraph()

    # Detectar los DataFrames
    df_names = [
        name for name in globals_dict
        if name.startswith('df_') and isinstance(globals_dict[name], pd.DataFrame)
    ]

    if not df_names:
        print("No se encontraron DataFrames.")
        return

    # Buscar relaciones reales basadas en columnas comunes
    for dfx_name in df_names:
        df1 = globals_dict[dfx_name]
        columnas_df1 = set(df1.columns)

        for df2_name in df_names:
            if df2_name != dfx_name:
                df2 = globals_dict[df2_name]
                columnas_df2 = set(df2.columns)
                columnas_comunes = columnas_df1 & columnas_df2

                if columnas_comunes:
                    G.add_edge(dfx_name, df2_name)
                    relaciones[f"{dfx_name} -> {df2_name}"] = list(columnas_comunes)

    # Eliminar ciclos detectados para evitar dependencias circulares
    try:
        ciclo = list(nx.simple_cycles(G))
        if ciclo:
            print(f"Ciclos detectados en el grafo: {ciclo}")
            G.remove_edges_from([(n1, n2) for n1, n2 in G.edges() if (n1, n2) in ciclo])
    except nx.NetworkXError as e:
        print(f"Error al eliminar ciclos: {e}")
    
    # Encontrar nodos raíz (sin predecesores)
    posibles_raices = [n for n in G.nodes if G.in_degree(n) == 0]

    if not posibles_raices:
        print("No se encontró un DataFrame raíz. El grafo tiene ciclos o todos tienen predecesores.")
        return

    # Si hay más de una raíz, tratamos de crear árboles por separado para cada una
    arborescencias = {}
    for root_df in posibles_raices:
        try:
            T = nx.bfs_tree(G, root_df)
            arborescencias[root_df] = T
        except nx.NetworkXError as e:
            print(f"Error al construir el árbol desde {root_df}: {e}")
            continue

    # Mostrar relaciones
    for rel, cols in relaciones.items():
        print(f"{rel}: columnas comunes -> {cols}")

    # Dibujar todas las arborescencias encontradas
    plt.figure(figsize=(15, 10))
    pos = nx.spring_layout(G, k=0.8)

    # Dibujar cada árbol raíz
    for root_df, T in arborescencias.items():
        nx.draw(T, pos, with_labels=True, node_size=4000, node_color='lightblue',
                font_size=10, font_weight='bold', edge_color='blue', arrows=True)
    
    plt.title("Arborescencia de Dependencias entre DataFrames")
    plt.show()

'''
    # ejemplo
df_A = pd.DataFrame({'id': [1, 2, 3], 'nombre': ['A', 'B', 'C']})
df_B = pd.DataFrame({'id': [1, 2], 'valor': [100, 200]})
df_C = pd.DataFrame({'id': [1], 'otro': ['X']})
df_D = pd.DataFrame({'nombre': ['A', 'B'], 'edad': [30, 40]})
tree_dataframes(globals())
'''

"\n    # ejemplo\ndf_A = pd.DataFrame({'id': [1, 2, 3], 'nombre': ['A', 'B', 'C']})\ndf_B = pd.DataFrame({'id': [1, 2], 'valor': [100, 200]})\ndf_C = pd.DataFrame({'id': [1], 'otro': ['X']})\ndf_D = pd.DataFrame({'nombre': ['A', 'B'], 'edad': [30, 40]})\ntree_dataframes(globals())\n"

## Funciones GPS

Aqui agrupamos las funciones de apoyo para temas de localización

### haversine
🔍 ¿Qué hace?

Calcula la distancia entre dos puntos en la superficie de la Tierra utilizando la fórmula de Haversine, que considera la curvatura del planeta (es decir, no asume una superficie plana como lo haría una distancia euclidiana).

Parámetros:

    🔹lat1, lon1: Latitud y longitud del primer punto (en grados decimales).

    🔹lat2, lon2: Latitud y longitud del segundo punto (también en grados decimales).

🧠 Explicación paso a paso:

    🔹 Radio de la Tierra (R): Se toma como 6371 km (valor promedio).

    🔹 Conversión a radianes: Las funciones trigonométricas de math usan radianes, así que se convierte cada latitud y longitud de grados a radianes.

    🔹 Diferencia angular: Calcula la diferencia entre latitudes y longitudes en radianes (dlat, dlon).

🎼 Fórmula de Haversine:

    🔹 Calcula el valor de a, que es una medida intermedia basada en el seno de las mitades de las diferencias angulares.

    🔹 Luego obtiene c, que es el ángulo central entre los dos puntos en una esfera.

    🔹 Distancia final: Multiplica el radio de la Tierra por c para obtener la distancia en kilómetros.

In [21]:
def haversine(lat1, lon1, lat2, lon2):
    # Radio de la Tierra en kilómetros
    R = 6371.0
    
    # Convertir de grados a radianes
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)
    
    # Diferencias de latitudes y longitudes
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    
    # Fórmula de Haversine
    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    # Distancia en kilómetros
    distance = R * c
    return distance

'''
# Ejemplo de uso
lat1, lon1 = 40.748817, -73.985428  # Coordenadas de ejemplo: New York (lat, lon)
lat2, lon2 = 34.052235, -118.243683  # Coordenadas de ejemplo: Los Angeles (lat, lon)

distancia = haversine(lat1, lon1, lat2, lon2)
print(f"La distancia entre las dos ubicaciones es: {distancia:.2f} km")
'''

'\n# Ejemplo de uso\nlat1, lon1 = 40.748817, -73.985428  # Coordenadas de ejemplo: New York (lat, lon)\nlat2, lon2 = 34.052235, -118.243683  # Coordenadas de ejemplo: Los Angeles (lat, lon)\n\ndistancia = haversine(lat1, lon1, lat2, lon2)\nprint(f"La distancia entre las dos ubicaciones es: {distancia:.2f} km")\n'

### location_gps
🔍 ¿Qué hace?

Intenta determinar la ubicación del dispositivo basándose en la IP.

🧠 ¿Qué hace paso a paso?

Llama a geocoder.ip('me'), que intenta determinar la ubicación del dispositivo basándose en la IP.

Extrae las coordenadas de latitud y longitud (latlng) del resultado.

Muestra las coordenadas por consola con print().

In [22]:
def location_gps():
    ubicacion = geocoder.ip('me')
    
    # Mostrar coordenadas
    print(f"Tu ubicación aproximada es: {ubicacion.latlng}")
'''
## ejemplo de ejecución 
location_gps()
'''

'\n## ejemplo de ejecución \nlocation_gps()\n'

In [15]:
convert_list ={'idflinea':'route_id','idfestacion':'station_id','idestacion':'station_id','idanden':'platform_id'}


convert_keys = [
    {'trip_id' : ['service_id', '0:5']},
    {'trip_id' : ['route_id', '5:9']}
    
]

convert_keys = []


#----------------------------------
# ### Inicialización de Variables
#----------------------------------


explore = 15 ## Parametro para delimitar el numero de elementos devueltos
split_key = 'df_M4' ## Raiz para la división entre dataframe 


def format_trip_id(row):
    route_id = row['route_id']
    line = row['idftramo']
    if pd.isnull(route_id):
        return None

    line = line.split(route_id)[1]
    return f"4_I{route_id}_2023I_{line}"

    try:
        # Quitamos el prefijo "4__" y separamos
        parts = route_id.replace('4__', '')  # ['12', '1', '', '']
        linea = parts[0]  # "12"
        if linea =='_':
            linea = parts[1] 
        print('linea',linea)
        sublinea = parts[1]  # "1"
        padded_sublinea = str(int(sublinea)).zfill(3)  # "001"

        return f"4_I{linea}-{padded_sublinea}_2023I_"

    except Exception:
        return f"4_I{route_id}_2023I_"